In [1]:
import polars as pl
import pyodbc
from datetime import date

### Definición de Rutas

In [2]:
cierre = "202505"
cierre_l = "May25"
cierre_d = date(2025,5,31)

# Carpetas
wd = "E:/Users/jhernandezr/DAR/garantias/reporte/fotos/"
wd_data_raw = wd + "data/raw/"
wd_data_external = wd + "data/external/"
wd_data_processed_dwh = "E:/Users/jhernandezr/DAR/garantias/data_pipeline_garantias/data/processed/dwh/"
wd_data_processed_fotos = "E:/Users/jhernandezr/DAR/garantias/data_pipeline_garantias/data/processed/fotos/"

# Inputs
fl_catalogos = wd_data_external + f'Catálogos_{cierre_l}'
fl_recuperadas_detalle = wd_data_processed_dwh + f'parquet/Recupera_con_Pagos_Flujos_{cierre}.parquet'
fl_pagadas_detalle = wd_data_processed_dwh + f'parquet/Pagadas_Global_VF_{cierre}.parquet'

# Outputs


### Importa tablas

In [3]:
schema_recuperadas = {
    'Concatenar_Saldos': pl.String,
    'Fecha_Consulta': pl.Datetime,
    'Programa_Id': pl.Int32,
    'Tipo_Garantia_Id': pl.Int16,
    'Tipo_Cambio_Cierre': pl.Float32,
    'Programa_Original': pl.Int32,
    'Porcentaje_Garantizado': pl.Float32,
    'Monto _Credito_Mn': pl.Float64,
    'Fecha_Apertura': pl.Datetime,
    'Moneda_Id': pl.Int16,
    'Tipo_Credito_Id': pl.Int16,
    'Fecha_Garantia_Honrada': pl.Datetime,
    'TPRO_CLAVE': pl.Int32,
    'NR_R': pl.String,
    'Producto': pl.String,
    'AGRUPAMIENTO_ID': pl.Float64,
    'AGRUPAMIENTO': pl.String,
    'Intermediario_Id': pl.String,
    'Numero_Credito': pl.String,
    'Id': pl.Int16,
    'Monto': pl.Float64,
    'Interes': pl.Float64,
    'Moratorios': pl.Float64,
    'Descripcion': pl.String,
    'Estatus': pl.String,
    'Fecha_Registro': pl.Datetime,
    'Fecha': pl.Datetime,
    'Monto_Mn': pl.Float64,
    'Interes_Mn': pl.Float64,
    'Excedente': pl.Float64,
    'Excedente_Mn': pl.Float64,
    'Moratorios_Mn': pl.Float64,
    'Sub_Total_Mn': pl.Float64,
    'RazÃ³n Social (Intermediario)': pl.String,
    'Empresa / Acreditado (DescripciÃ³n)': pl.String,
    'RFC Empresa / Acreditado': pl.String,
    'TIPO_PERSONA': pl.String,
    'Recup_Rescat': pl.String,
    'MM_UDIS': pl.Int64,
    'ESQUEMA': pl.String,
    'CSG': pl.String,
    'CSF': pl.String,
    'Gastos_Juicio_Mn': pl.Float64,
    'Monto_Total_Mn': pl.Float64,
    'MPagado_Mn': pl.Float64,
    'Historico': pl.String,
    'Fecha Registro Alta': pl.Datetime}

schema_pagadas = {
    'Concatenado': pl.String,
    'Fecha_Consulta': pl.Datetime,
    'Intermediario_Id': pl.String,
    'Numero_Credito': pl.String,
    'Producto': pl.String,
    'Pago ID': pl.Int16,
    'RazÃ³n Social (Intermediario)': pl.String,
    'MIN_Fecha_Registro': pl.Datetime,
    'Fecha_Garantia_Honrada': pl.Datetime,
    'TPRO_CLAVE': pl.Float32,
    'Programa_Original': pl.Int32,
    'Programa_Id': pl.Int32,
    'Monto _Credito_Mn': pl.Float64,
    'Moneda_Id': pl.Int16,
    'Fecha de Apertura': pl.Datetime,
    'Tipo_Garantia_Id': pl.Int16,
    'TIPO_PERSONA': pl.String,
    'RFC Empresa / Acreditado': pl.String,
    'Monto_Desembolsado': pl.Float64,
    'Interes_Desembolso': pl.Float64,
    'Interes_Moratorios': pl.Float64,
    'Porcentaje_Garantizado': pl.Float32,
    'Tipo_Credito_Id': pl.Int16,
    'Estatus_Recuperacion': pl.String,
    'Empresa / Acreditado (DescripciÃ³n)': pl.String,
    'Fecha Registro Alta': pl.Datetime,
    'TC': pl.Float32,
    'AGRUPAMIENTO_ID': pl.Float32, # Revisar
    'AGRUPAMIENTO': pl.String,
    'ESQUEMA': pl.String,
    'SUBESQUEMA': pl.String,
    'CAMBIO': pl.Float64,
    'MM_UDIS': pl.Int16,
    'NR_R': pl.String,
    'CSG': pl.String,
    'CSF': pl.String,
    'Monto_Desembolso_Mn': pl.Float64,
    'Interes_Desembolso_Mn': pl.Float64,
    'Interes_Moratorios_Mn': pl.Float64,
    'Monto_Pagado_Mn': pl.Float64
}

In [4]:
# Importa Recuperadas
recuperadas_detalle = pl.read_parquet(fl_recuperadas_detalle)
""" recuperadas_detalle = (recuperadas_detalle
                        .filter(
                        (pl.col("Producto") != "GARANTIAS BANCOMEXT") &
                        (pl.col("Producto") != "GARANTIAS SHF/LI FINANCIERO") &
                        (pl.col("Producto") != "GARANTIAS BANSEFI")
                        )) """

' recuperadas_detalle = (recuperadas_detalle\n                        .filter(\n                        (pl.col("Producto") != "GARANTIAS BANCOMEXT") &\n                        (pl.col("Producto") != "GARANTIAS SHF/LI FINANCIERO") &\n                        (pl.col("Producto") != "GARANTIAS BANSEFI")\n                        )) '

In [5]:
# Importa Pagadas
pagadas_detalle = pl.read_parquet(fl_pagadas_detalle)
""" pagadas_detalle = (pagadas_detalle
                   .filter(
                        (pl.col("Producto") != "GARANTIAS BANCOMEXT") &
                        (pl.col("Producto") != "GARANTIAS SHF/LI FINANCIERO") &
                        (pl.col("Producto") != "GARANTIAS BANSEFI")
                   ))
pagadas_detalle.shape """

' pagadas_detalle = (pagadas_detalle\n                   .filter(\n                        (pl.col("Producto") != "GARANTIAS BANCOMEXT") &\n                        (pl.col("Producto") != "GARANTIAS SHF/LI FINANCIERO") &\n                        (pl.col("Producto") != "GARANTIAS BANSEFI")\n                   ))\npagadas_detalle.shape '

In [6]:
print(recuperadas_detalle.shape)
print(pagadas_detalle.shape)

(1566470, 47)
(514042, 40)


In [7]:
# Importa Catálogos
conn_str = r"DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=" + fl_catalogos + '.accdb'
conn = pyodbc.connect(conn_str)
estatus = pl.read_database(query='SELECT * FROM [ESTATUS]', connection=conn)
llave = pl.read_database(query='SELECT * FROM [LLAVE]', connection=conn, schema_overrides={'Programa Mezcla ID': pl.Int32})
conn.close()



In [8]:
recuperadas_con_llave = recuperadas_detalle.join(
    llave.select([
            "Programa Mezcla ID",
            "Intermediario_Id",
            "LLAVE_FINAL"]),
        left_on=["TPRO_CLAVE", "Intermediario_Id"],
        right_on=["Programa Mezcla ID", "Intermediario_Id"],
        how="left")


In [9]:
pagadas_con_llave = pagadas_detalle.join(
    llave.select([
            "Programa Mezcla ID",
            "Intermediario_Id",
            "LLAVE_FINAL"]),
        left_on=["TPRO_CLAVE", "Intermediario_Id"],
        right_on=["Programa Mezcla ID", "Intermediario_Id"],
        how="left")

In [10]:
# RECUP_PREVIO_202504
def base_recup_detalle(recuperadas, estatus):
    # Se realiza el Join
    df = recuperadas.join(
            estatus.select(["Estatus ID", "Estatus"]),
            left_on="Estatus",
            right_on="Estatus ID",
            how="left"
    )
    df = df.with_columns([
        (pl.col("Monto_Mn").fill_null(0) / 1_000_000).alias("CAPRECUP(MDP)"),
        (pl.col("Interes_Mn").fill_null(0) / 1_000_000).alias("INTRECUP(MDP)"),
        (pl.col("Moratorios_Mn").fill_null(0) / 1_000_000).alias("MORAT(MDP)"),
        (pl.col("Excedente_Mn").fill_null(0) / 1_000_000).alias("EXCEDENTE(MDP)"),
        (pl.col("Gastos_Juicio_Mn").fill_null(0) / 1_000_000).alias("GASTOSJUICIO(MDP)"),
        (pl.col("Sub_Total_Mn").fill_null(0) / 1_000_000).alias("SUBTOTAL (MDP)"),
        (pl.col("Monto_Total_Mn").fill_null(0) / 1_000_000).alias("MONTOTOTAL (MDP)"),
        (pl.when(pl.col("Estatus") == "E").then(pl.lit("E")).otherwise(pl.lit("NE")).alias("REEST")),
        pl.col("Estatus_right").alias("STATUS_RECUP"),
        pl.when(pl.col("Fecha") > pl.col("Fecha_Garantia_Honrada")).then(1).otherwise(0).alias("IND_ENTRA"),
        
        # MRECUP_TOT(MDP)
        pl.when(pl.col("Estatus") == "Recuperada").then(
            (
                pl.col("Monto_Mn").fill_null(0) +
                pl.col("Interes_Mn").fill_null(0) +
                pl.col("Moratorios_Mn").fill_null(0) +
                pl.col("Excedente_Mn").fill_null(0) -
                pl.col("Gastos_Juicio_Mn").fill_null(0)
            ) / 1_000_000
        ).otherwise(0).alias("MRECUP_TOT(MDP)"),
        
        # MRESCAT_TOT(MDP)
        pl.when(pl.col("Estatus") == "Rescatada").then(
            (
                pl.col("Monto_Mn").fill_null(0) +
                pl.col("Interes_Mn").fill_null(0) +
                pl.col("Moratorios_Mn").fill_null(0) +
                pl.col("Excedente_Mn").fill_null(0) -
                pl.col("Gastos_Juicio_Mn").fill_null(0)
            ) / 1_000_000
        ).otherwise(0).alias("MRESCAT_TOT(MDP)"),

        # Diferencia en meses entre fechas
        ((pl.col("Fecha").dt.year() - pl.col("Fecha_Garantia_Honrada").dt.year()) * 12 +
        (pl.col("Fecha").dt.month() - pl.col("Fecha_Garantia_Honrada").dt.month())).alias("DIFERENCIAS_MESES"),

        pl.col("Fecha_Garantia_Honrada").dt.year().alias("ANIO_PAGO"),
        (pl.col("Fecha_Garantia_Honrada").dt.year() * 100 + pl.col("Fecha_Garantia_Honrada").dt.month()).alias("ORDEN_PAGO"),

        # Concatenación de campos como string
        (pl.col("Numero_Credito").cast(str) + pl.col("Intermediario_Id").cast(str)).alias("CONCATENAR_SALDOS")
    ])

    return df

recup_previo = base_recup_detalle(recuperadas_con_llave, estatus)

In [11]:
# CICLOS_RESCATE
def gen_ciclos_rescate(df):
    df_filtered = df.filter(
        pl.col("Numero_Credito").is_not_null() & 
        pl.col("Razón Social (Intermediario)").is_not_null()
    )

    # Agrupar y sumar
    agg_df = (
        df_filtered
        .group_by(["Numero_Credito", "Razón Social (Intermediario)", "CONCATENAR_SALDOS"])
        .agg(
            (pl.col("MRESCAT_TOT(MDP)") * pl.col("IND_ENTRA")).sum().alias("MRESCAT_TOTAL")
        )
    )

    # Agregar columnas requeridas
    final_df = (
        agg_df.with_columns([
            pl.col("Numero_Credito").alias("Prestamo"),
            pl.col("Razón Social (Intermediario)").alias("INTERMEDIARIO"),
            pl.when(pl.col("MRESCAT_TOTAL") == 0).then(1).otherwise(0).alias("CICLOS_RESCAT")
        ])
        .select([
            "CONCATENAR_SALDOS", 
            "Prestamo", 
            "INTERMEDIARIO", 
            "MRESCAT_TOTAL", 
            "CICLOS_RESCAT"
        ])
    )
    return final_df

ciclos_rescate = gen_ciclos_rescate(recup_previo)

#### Parte 2

In [12]:
# CURVA_RECUP_202504
curva_recup = recup_previo.join(
    ciclos_rescate.select(["CONCATENAR_SALDOS", "CICLOS_RESCAT"]),
    on="CONCATENAR_SALDOS",
    how="left"
)

In [13]:
curva_recup = curva_recup.with_columns([
    pl.col("Fecha_Registro").dt.year().alias("Anio_REG_RECUP"),
    pl.col("Fecha_Registro").dt.month().alias("Mes_REG_RECUP")
])

# Agrupar y sumar
# RECUP_AGRUP_202504_inter
recup_agrup_inter = (
    curva_recup
    .group_by([
        "IND_ENTRA", "MM_UDIS", "Producto", "NR_R", "LLAVE_FINAL", "TPRO_CLAVE",
        "Razón Social (Intermediario)", "CSG", "CSF", "AGRUPAMIENTO", "AGRUPAMIENTO_ID",
        "Intermediario_Id", "CICLOS_RESCAT", "DIFERENCIAS_MESES", "Tipo_Garantia_Id",
        "ANIO_PAGO", "REEST", "Anio_REG_RECUP", "ESQUEMA", "ORDEN_PAGO"
    ])
    .agg([
        pl.col("MRECUP_TOT(MDP)").sum().alias("MRECUP_TOT"),
        pl.col("MRESCAT_TOT(MDP)").sum().alias("MRESCAT_TOT")
    ])
    .rename({
        "Producto": "TAXONOMIA",
        "Tipo_Garantia_Id": "TIPGAR_CLAVE",
        "ANIO_PAGO": "ANIO(PAGO)",
        "Razón Social (Intermediario)": "BANCO"
    })
)
recup_agrup_inter

IND_ENTRA,MM_UDIS,TAXONOMIA,NR_R,LLAVE_FINAL,TPRO_CLAVE,BANCO,CSG,CSF,AGRUPAMIENTO,AGRUPAMIENTO_ID,Intermediario_Id,CICLOS_RESCAT,DIFERENCIAS_MESES,TIPGAR_CLAVE,ANIO(PAGO),REEST,Anio_REG_RECUP,ESQUEMA,ORDEN_PAGO,MRECUP_TOT,MRESCAT_TOT
i32,i32,str,str,str,i32,str,str,str,str,f64,str,i32,i32,i32,i32,str,i32,str,i32,f64,f64
1,0,"""GARANTIA EMPRESARIAL""","""R""","""CREDIACTIVO DIANA""",31028,"""BANCO MERCANTIL DEL NORTE""","""SG""","""CF""","""Crediactivo Diana""",1275.0,"""10040072""",1,27,5,2012,"""NE""",2015,"""PP""",201212,0.0,0.0
1,0,"""GARANTIA EMPRESARIAL""","""R""","""CREDITO A NEGOCIOS""",3851,"""HSBC MEXICO""","""SG""","""SF""","""Montos Menores Self Employ""",1635.0,"""10000171""",1,150,5,2011,"""NE""",2024,"""PP""",201106,0.0,0.0
1,0,"""GARANTIA SUBASTA""","""R""","""MICROCREDITO""",3338,"""BANAMEX""","""SG""","""CF""","""Subasta Extraordinaria""",1750.0,"""10000233""",1,44,5,2011,"""NE""",2014,"""1P""",201104,0.0,0.0
1,0,"""GARANTIA EMPRESARIAL""","""R""",null,35709,"""BANAMEX""","""SG""","""CF""","""PyMe""",1690.0,"""10000233""",1,41,5,2019,"""NE""",2022,"""PP""",201905,0.0,0.0
1,0,"""GARANTIA EMPRESARIAL""","""NR""","""CREDITO PYME""",35175,"""BANCO SANTANDER""","""SG""","""CF""","""PyMe""",1690.0,"""10000144""",1,5,4,2015,"""NE""",2016,"""PP""",201508,0.0,0.0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
1,0,"""GARANTIA EMERGENTES""","""NR""","""CREDITO PYME""",31015,"""BANCO SANTANDER""","""SG""","""CF""","""Emergente Contingencia Sanitar…",1390.0,"""10000144""",1,48,4,2011,"""NE""",2015,"""PP""",201105,0.0,0.0
1,0,"""GARANTIA SECTORIAL""","""NR""","""TRADICIONAL""",3268,"""BANCA MIFEL""","""SG""","""CF""","""Taxi DF""",1930.0,"""10000420""",1,2,4,2011,"""NE""",2012,"""PP""",201112,0.0,0.0
1,0,"""GARANTIA EMPRESARIAL""","""R""","""CREDITO A NEGOCIOS""",32907,"""HSBC MEXICO""","""SG""","""CF""","""Micro Empresa PM""",1595.0,"""10000171""",1,2,5,2016,"""NE""",2017,"""PP""",201611,0.0,0.0


In [14]:
# Agrupar y agregar
# RECUP_AGRUP_202504_inter_Mes
recup_agrup_inter_mes = (
    curva_recup
    .group_by([
        "IND_ENTRA", "MM_UDIS", "Producto", "NR_R", "LLAVE_FINAL", "TPRO_CLAVE",
        "Razón Social (Intermediario)", "CSG", "CSF", "AGRUPAMIENTO", "AGRUPAMIENTO_ID",
        "Intermediario_Id", "CICLOS_RESCAT", "DIFERENCIAS_MESES", "Tipo_Garantia_Id",
        "ANIO_PAGO", "REEST", "Anio_REG_RECUP", "Mes_REG_RECUP", "ESQUEMA", "ORDEN_PAGO"
    ])
    .agg([
        pl.col("MRECUP_TOT(MDP)").sum().alias("MRECUP_TOT"),
        pl.col("MRESCAT_TOT(MDP)").sum().alias("MRESCAT_TOT")
    ])
    .rename({
        "Producto": "TAXONOMIA",
        "Tipo_Garantia_Id": "TIPGAR_CLAVE",
        "ANIO_PAGO": "ANIO(PAGO)",
        "Razón Social (Intermediario)": "BANCO"
    })
)
recup_agrup_inter_mes

IND_ENTRA,MM_UDIS,TAXONOMIA,NR_R,LLAVE_FINAL,TPRO_CLAVE,BANCO,CSG,CSF,AGRUPAMIENTO,AGRUPAMIENTO_ID,Intermediario_Id,CICLOS_RESCAT,DIFERENCIAS_MESES,TIPGAR_CLAVE,ANIO(PAGO),REEST,Anio_REG_RECUP,Mes_REG_RECUP,ESQUEMA,ORDEN_PAGO,MRECUP_TOT,MRESCAT_TOT
i32,i32,str,str,str,i32,str,str,str,str,f64,str,i32,i32,i32,i32,str,i32,i8,str,i32,f64,f64
1,0,"""GARANTIA EMPRESARIAL""","""NR""",null,35660,"""HSBC MEXICO""","""SG""","""CF""","""PYME Negocios""",2080.0,"""10000171""",1,56,4,2019,"""NE""",2024,5,"""PP""",201904,0.0,0.0
1,0,"""GARANTIA EMPRESARIAL""","""NR""","""CREDITO SIMPLE""",35321,"""BBV BANCOMER""","""SG""","""CF""","""PyMe""",1690.0,"""10040012""",1,30,4,2018,"""NE""",2020,12,"""PP""",201806,0.0,0.0
1,0,"""GARANTIA SECTORIAL""","""NR""",null,36535,"""BANCO SANTANDER""","""SG""","""CF""","""Mujer PYME""",1645.0,"""10000144""",1,39,4,2019,"""NE""",2022,10,"""PP""",201907,0.0,0.0
1,0,"""GARANTIAS BANCOMEXT""","""NR""",null,39620,"""BANCOMEXTBANAMEX""","""SG""","""CF""","""Bancomext""",1225.0,"""468297752""",1,22,5,2020,"""NE""",2022,8,"""PP""",202010,0.0,0.0
1,0,"""GARANTIA EMPRESARIAL""","""NR""","""CREDITO PYME""",35115,"""BANCO SANTANDER""","""SG""","""CF""","""PyMe""",1690.0,"""10000144""",1,12,4,2015,"""NE""",2016,2,"""PP""",201502,0.0,0.0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
1,0,"""GARANTIA EMPRESARIAL""","""NR""","""CREDITO PYME""",35201,"""BANCO SANTANDER""","""SG""","""CF""","""PyMe""",1690.0,"""10000144""",1,5,4,2017,"""NE""",2017,12,"""PP""",201707,0.0,0.0
1,0,"""GARANTIA SECTORIAL""","""NR""",null,37066,"""BANCO SANTANDER""","""SG""","""CF""","""Impulso Económico""",1575.0,"""10000144""",1,17,4,2020,"""NE""",2022,3,"""PP""",202010,0.0,0.0
1,0,"""GARANTIA EMPRESARIAL""","""R""","""CREDITO LIQUIDO""",3198,"""BBV BANCOMER""","""SG""","""CF""","""Paraguas""",1545.0,"""10040012""",1,11,4,2011,"""NE""",2012,8,"""PP""",201108,0.0,0.0


In [ ]:
# RECUP_AGRUP_202504_Completa
recup_agrup_inter_completa = recup_agrup_inter.with_columns([
    pl.when(pl.col("ESQUEMA") == "PP")
      .then(pl.lit("Pari Passu"))
      .when(pl.col("ESQUEMA").is_null())
      .then(pl.lit("Pari Passu"))
      .otherwise(pl.lit("1P"))
      .alias("ESQUEMA_VF"),

    pl.when(pl.col("ANIO(PAGO)") <= 2006)
      .then(2006)
      .otherwise(pl.col("ANIO(PAGO)"))
      .alias("ANIO(PAGO)_New")
])

In [16]:
# RECUP_AGRUP_202504_Completa_Mes
recup_agrup_inter_mes = recup_agrup_inter_mes.with_columns([
    pl.when(pl.col("ESQUEMA") == "PP")
      .then(pl.lit("Pari Passu"))
      .when(pl.col("ESQUEMA").is_null())
      .then(pl.lit("Pari Passu"))
      .otherwise(pl.lit("1P"))
      .alias("ESQUEMA_VF"),

    pl.when(pl.col("ANIO(PAGO)") <= 2006)
      .then(2006)
      .otherwise(pl.col("ANIO(PAGO)"))
      .alias("ANIO(PAGO)_New")
])

In [ ]:
# RECUP_AGRUP_202504
recup_agrup = recup_agrup_inter_completa.select([
    "IND_ENTRA",
    "MM_UDIS",
    "TAXONOMIA",
    "NR_R",
    "BANCO",
    "CSG",
    "CSF",
    "AGRUPAMIENTO",
    "AGRUPAMIENTO_ID",
    "CICLOS_RESCAT",
    "DIFERENCIAS_MESES",
    "TIPGAR_CLAVE",
    "ANIO(PAGO)",
    "REEST",
    "MRECUP_TOT",
    "MRESCAT_TOT",
    "Anio_REG_RECUP",
    pl.col("ESQUEMA_VF").alias("ESQUEMA"),
    "ANIO(PAGO)_New",
    "Intermediario_Id",
    "TPRO_CLAVE",
    "ORDEN_PAGO"
])
recup_agrup

IND_ENTRA,MM_UDIS,TAXONOMIA,NR_R,BANCO,CSG,CSF,AGRUPAMIENTO,AGRUPAMIENTO_ID,CICLOS_RESCAT,DIFERENCIAS_MESES,TIPGAR_CLAVE,ANIO(PAGO),REEST,MRECUP_TOT,MRESCAT_TOT,Anio_REG_RECUP,ESQUEMA,ANIO(PAGO)_New,Intermediario_Id,TPRO_CLAVE,ORDEN_PAGO
i32,i32,str,str,str,str,str,str,f64,i32,i32,i32,i32,str,f64,f64,i32,str,i32,str,i32,i32
1,0,"""GARANTIA EMPRESARIAL""","""R""","""BANCO MERCANTIL DEL NORTE""","""SG""","""CF""","""Crediactivo Diana""",1275.0,1,27,5,2012,"""NE""",0.0,0.0,2015,"""Pari Passu""",2012,"""10040072""",31028,201212
1,0,"""GARANTIA EMPRESARIAL""","""R""","""HSBC MEXICO""","""SG""","""SF""","""Montos Menores Self Employ""",1635.0,1,150,5,2011,"""NE""",0.0,0.0,2024,"""Pari Passu""",2011,"""10000171""",3851,201106
1,0,"""GARANTIA SUBASTA""","""R""","""BANAMEX""","""SG""","""CF""","""Subasta Extraordinaria""",1750.0,1,44,5,2011,"""NE""",0.0,0.0,2014,"""1P""",2011,"""10000233""",3338,201104
1,0,"""GARANTIA EMPRESARIAL""","""R""","""BANAMEX""","""SG""","""CF""","""PyMe""",1690.0,1,41,5,2019,"""NE""",0.0,0.0,2022,"""Pari Passu""",2019,"""10000233""",35709,201905
1,0,"""GARANTIA EMPRESARIAL""","""NR""","""BANCO SANTANDER""","""SG""","""CF""","""PyMe""",1690.0,1,5,4,2015,"""NE""",0.0,0.0,2016,"""Pari Passu""",2015,"""10000144""",35175,201508
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
1,0,"""GARANTIA EMERGENTES""","""NR""","""BANCO SANTANDER""","""SG""","""CF""","""Emergente Contingencia Sanitar…",1390.0,1,48,4,2011,"""NE""",0.0,0.0,2015,"""Pari Passu""",2011,"""10000144""",31015,201105
1,0,"""GARANTIA SECTORIAL""","""NR""","""BANCA MIFEL""","""SG""","""CF""","""Taxi DF""",1930.0,1,2,4,2011,"""NE""",0.0,0.0,2012,"""Pari Passu""",2011,"""10000420""",3268,201112
1,0,"""GARANTIA EMPRESARIAL""","""R""","""HSBC MEXICO""","""SG""","""CF""","""Micro Empresa PM""",1595.0,1,2,5,2016,"""NE""",0.0,0.0,2017,"""Pari Passu""",2016,"""10000171""",32907,201611


In [18]:
# TABLA_ULT_PAGO
ult_pago = (
    pagadas_con_llave.group_by(["Intermediario_Id", "Numero_Credito"])
      .agg(pl.col("Pago ID").max().alias("ULT_PAGO"))
)
ult_pago


Intermediario_Id,Numero_Credito,ULT_PAGO
str,str,i64
"""465666078""","""30251539""",1
"""248156063""","""180596798""",1
"""248156063""","""124999280""",1
"""10000233""","""RN88110800019565""",1
"""248156063""","""175792219""",1
…,…,…
"""248156063""","""200538865""",1
"""10000171""","""79332311""",1
"""248156063""","""155356602""",1


In [19]:
# RECUP_PREVIO_202504_cUP
def gen_recup_previo_cup(pagadas_con_llave, ult_pago):
    df_joined = pagadas_con_llave.join(
        ult_pago,
        on=["Intermediario_Id", "Numero_Credito"],
        how="left"
    )

    # Filtrar donde 'Pago ID' == 'ULT_PAGO' y ULT_PAGO no es null
    df_result = (
        df_joined.filter(
            (pl.col("Pago ID") == pl.col("ULT_PAGO")) &
            (pl.col("ULT_PAGO").is_not_null())
        )
    )
    return df_result
recup_previo_cup = gen_recup_previo_cup(pagadas_con_llave, ult_pago)

In [20]:
# PAGADAS_PREVIO_202504
pagadas_previo = (
    recup_previo_cup.with_columns([
        (pl.col("Monto_Pagado_Mn").fill_null(0) / 1_000_000).alias("MPAGADO (MDP)"),
        pl.col("Fecha_Garantia_Honrada").dt.year().alias("ANIO_PAGO"),
        (
            pl.col("Fecha_Garantia_Honrada").dt.year() * 100 +
            pl.col("Fecha_Garantia_Honrada").dt.month()
        ).alias("ORDEN_PAGO"),
        (pl.col("Numero_Credito").cast(str) + pl.col("Intermediario_Id").cast(str)).alias("CONCATENAR_SALDOS")
    ])
)


#### Parte 3

In [21]:
# PAGADAS_DETALLE_VF_202504
pagadas_detalle_vf = pagadas_previo.join(
    ciclos_rescate.select(["CONCATENAR_SALDOS", "CICLOS_RESCAT"]),
    on="CONCATENAR_SALDOS",
    how="left"
)
pagadas_detalle_vf.shape

(513901, 47)

In [22]:
#PAGADAS_AGRUP_202504_inter
pagadas_agrup_inter = (
    pagadas_detalle_vf.group_by([
        "MM_UDIS", "Producto", "NR_R", "LLAVE_FINAL", "TPRO_CLAVE",
        "Razón Social (Intermediario)", "CSG", "CSF", "AGRUPAMIENTO",
        "AGRUPAMIENTO_ID", "Intermediario_Id", "CICLOS_RESCAT",
        "Tipo_Garantia_Id", "ANIO_PAGO", "ESQUEMA", "ORDEN_PAGO"
    ])
    .agg([
        pl.col("MPAGADO (MDP)").sum().alias("MPAGADO")
    ])
    .rename({
        "Producto": "TAXONOMIA",
        "Razón Social (Intermediario)": "BANCO",
        "Tipo_Garantia_Id": "TIPGAR_CLAVE",
        "ANIO_PAGO": "ANIO(PAGO)"
    })
)
pagadas_agrup_inter.shape

(51650, 17)

In [23]:
# PAGADAS_AGRUP_202504_Completa
pagadas_agrup_completa = pagadas_agrup_inter.with_columns([
    # CICLOS_RESCAT_VF: 1 si es nulo, 0 si es 0, 1 si no es 0
    pl.when(pl.col("CICLOS_RESCAT").is_null())
        .then(1)
        .when(pl.col("CICLOS_RESCAT") == 0)
        .then(0)
        .otherwise(1)
        .alias("CICLOS_RESCAT_VF"),

    # ESQUEMA_VF: 'Pari Passu' si ESQUEMA es 'PP' o null, sino '1P'
    pl.when(pl.col("ESQUEMA") == 'PP')
        .then(pl.lit("Pari Passu"))
        .when(pl.col("ESQUEMA").is_null())
        .then(pl.lit("Pari Passu"))
        .otherwise(pl.lit("1P"))
        .alias("ESQUEMA_VF"),

    # ANIO(PAGO)_New: si ANIO(PAGO) <= 2006, entonces 2006
    pl.when(pl.col("ANIO(PAGO)") <= 2006)
        .then(2006)
        .otherwise(pl.col("ANIO(PAGO)"))
        .alias("ANIO(PAGO)_New")
])
pagadas_agrup_completa


MM_UDIS,TAXONOMIA,NR_R,LLAVE_FINAL,TPRO_CLAVE,BANCO,CSG,CSF,AGRUPAMIENTO,AGRUPAMIENTO_ID,Intermediario_Id,CICLOS_RESCAT,TIPGAR_CLAVE,ANIO(PAGO),ESQUEMA,ORDEN_PAGO,MPAGADO,CICLOS_RESCAT_VF,ESQUEMA_VF,ANIO(PAGO)_New
i32,str,str,str,i32,str,str,str,str,f64,str,i32,i16,i32,str,i32,f64,i32,str,i32
0,"""GARANTIA SECTORIAL""","""NR""",null,320132,"""BANAMEX""","""SG""","""CF""","""Mujer Crezcamos Juntas""",2045.0,"""10000233""",1,5,2022,"""PP""",202202,-0.159818,1,"""Pari Passu""",2022
0,"""GARANTIA EMPRESARIAL""","""NR""","""CREDITO PYME""",31759,"""BANCO SANTANDER""","""SG""","""CF""","""PyMe""",1690.0,"""10000144""",1,4,2015,"""PP""",201506,-0.038288,1,"""Pari Passu""",2015
0,"""GARANTIA SECTORIAL""","""NR""",null,320512,"""BANCO SANTANDER""","""SG""","""CF""","""Impulso NAFIN""",2195.0,"""10000144""",null,4,2024,"""PP""",202402,-0.020209,1,"""Pari Passu""",2024
0,"""GARANTIA EMPRESARIAL""","""NR""",null,35606,"""BANCO SANTANDER""","""SG""","""CF""","""PyMe""",1690.0,"""10000144""",1,4,2019,"""PP""",201906,-0.594916,1,"""Pari Passu""",2019
0,"""GARANTIA EMPRESARIAL""","""NR""","""CREDITO SIMPLE""",35005,"""BBV BANCOMER""","""SG""","""CF""","""PyMe""",1690.0,"""10040012""",1,4,2015,"""PP""",201511,-0.146386,1,"""Pari Passu""",2015
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
0,"""GARANTIA SECTORIAL""","""NR""",null,33760,"""BANCO MERCANTIL DEL NORTE""","""SG""","""CF""","""ZEE""",2050.0,"""10040072""",null,4,2018,"""PP""",201803,-0.500346,1,"""Pari Passu""",2018
0,"""GARANTIA EMPRESARIAL""","""R""",null,310020,"""BANCO REGIONAL DE MONTERREY""","""SG""","""SF""","""Producto NAFIN""",10000.0,"""10040058""",null,4,2022,"""PP""",202202,-1.270542,1,"""Pari Passu""",2022
1,"""GARANTIA EMPRESARIAL""","""NR""",null,310163,"""BBV BANCOMER""","""SG""","""SF""","""Producto NAFIN""",10000.0,"""10040012""",null,4,2025,"""PP""",202503,-8.545175,1,"""Pari Passu""",2025


In [24]:
# PAGADAS_AGRUP_202504
pagadas_agrup = pagadas_agrup_completa.select([
    "MM_UDIS",
    "TAXONOMIA",
    "NR_R",
    "BANCO",
    "CSG",
    "CSF",
    "AGRUPAMIENTO",
    "AGRUPAMIENTO_ID",
    pl.col("CICLOS_RESCAT_VF").alias("CICLOS_RESCAT"),
    "TIPGAR_CLAVE",
    "ANIO(PAGO)",
    "MPAGADO",
    pl.col("ESQUEMA_VF").alias("ESQUEMA"),
    "ANIO(PAGO)_New",
    "Intermediario_Id",
    "TPRO_CLAVE",
    "ORDEN_PAGO"
])
pagadas_agrup

MM_UDIS,TAXONOMIA,NR_R,BANCO,CSG,CSF,AGRUPAMIENTO,AGRUPAMIENTO_ID,CICLOS_RESCAT,TIPGAR_CLAVE,ANIO(PAGO),MPAGADO,ESQUEMA,ANIO(PAGO)_New,Intermediario_Id,TPRO_CLAVE,ORDEN_PAGO
i32,str,str,str,str,str,str,f64,i32,i16,i32,f64,str,i32,str,i32,i32
0,"""GARANTIA SECTORIAL""","""NR""","""BANAMEX""","""SG""","""CF""","""Mujer Crezcamos Juntas""",2045.0,1,5,2022,-0.159818,"""Pari Passu""",2022,"""10000233""",320132,202202
0,"""GARANTIA EMPRESARIAL""","""NR""","""BANCO SANTANDER""","""SG""","""CF""","""PyMe""",1690.0,1,4,2015,-0.038288,"""Pari Passu""",2015,"""10000144""",31759,201506
0,"""GARANTIA SECTORIAL""","""NR""","""BANCO SANTANDER""","""SG""","""CF""","""Impulso NAFIN""",2195.0,1,4,2024,-0.020209,"""Pari Passu""",2024,"""10000144""",320512,202402
0,"""GARANTIA EMPRESARIAL""","""NR""","""BANCO SANTANDER""","""SG""","""CF""","""PyMe""",1690.0,1,4,2019,-0.594916,"""Pari Passu""",2019,"""10000144""",35606,201906
0,"""GARANTIA EMPRESARIAL""","""NR""","""BBV BANCOMER""","""SG""","""CF""","""PyMe""",1690.0,1,4,2015,-0.146386,"""Pari Passu""",2015,"""10040012""",35005,201511
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
0,"""GARANTIA SECTORIAL""","""NR""","""BANCO MERCANTIL DEL NORTE""","""SG""","""CF""","""ZEE""",2050.0,1,4,2018,-0.500346,"""Pari Passu""",2018,"""10040072""",33760,201803
0,"""GARANTIA EMPRESARIAL""","""R""","""BANCO REGIONAL DE MONTERREY""","""SG""","""SF""","""Producto NAFIN""",10000.0,1,4,2022,-1.270542,"""Pari Passu""",2022,"""10040058""",310020,202202
1,"""GARANTIA EMPRESARIAL""","""NR""","""BBV BANCOMER""","""SG""","""SF""","""Producto NAFIN""",10000.0,1,4,2025,-8.545175,"""Pari Passu""",2025,"""10040012""",310163,202503


#### Parte 4

In [25]:
# CURVA_RECUP_202504_VF
curva_recup_vf = curva_recup.join(
    pagadas_detalle_vf.select(["CONCATENAR_SALDOS", "Monto_Pagado_Mn"]),
    on="CONCATENAR_SALDOS",
    how="left"
).with_columns(
    (pl.col("Monto_Pagado_Mn") / 1_000_000).alias("MPAGADO (MDP)")
)

In [26]:
# TABLA_SEV_OBS_202504
# Filtrar condiciones
tabla_sev_obs = recup_agrup.filter(
    (pl.col("CICLOS_RESCAT") == 1) &
    (pl.col("ANIO(PAGO)") <= 2018) &
    (pl.col("IND_ENTRA") == 1)
).select([   # Seleccionar y transformar columnas
    pl.col("MM_UDIS"),
    pl.col("TAXONOMIA"),
    pl.col("NR_R"),
    pl.col("BANCO"),
    pl.col("CSG"),
    pl.col("AGRUPAMIENTO"),
    pl.col("AGRUPAMIENTO_ID"),
    pl.col("TIPGAR_CLAVE"),
    pl.col("ESQUEMA"),
    pl.col("TPRO_CLAVE"),
    pl.col("ANIO(PAGO)_New"),
    pl.lit("Recup").alias("CONCEPTO"),
    pl.when(pl.col("Anio_REG_RECUP") <= 2018)
      .then(pl.lit("Inicio"))
      .otherwise(pl.lit("Acum"))
      .alias("PERIODO"),
    pl.col("MRECUP_TOT").alias("MONTO")
])


tabla_sev_obs

MM_UDIS,TAXONOMIA,NR_R,BANCO,CSG,AGRUPAMIENTO,AGRUPAMIENTO_ID,TIPGAR_CLAVE,ESQUEMA,TPRO_CLAVE,ANIO(PAGO)_New,CONCEPTO,PERIODO,MONTO
i32,str,str,str,str,str,f64,i32,str,i32,i32,str,str,f64
0,"""GARANTIA EMPRESARIAL""","""R""","""BANCO MERCANTIL DEL NORTE""","""SG""","""Crediactivo Diana""",1275.0,5,"""Pari Passu""",31028,2012,"""Recup""","""Inicio""",0.0
0,"""GARANTIA EMPRESARIAL""","""R""","""HSBC MEXICO""","""SG""","""Montos Menores Self Employ""",1635.0,5,"""Pari Passu""",3851,2011,"""Recup""","""Acum""",0.0
0,"""GARANTIA SUBASTA""","""R""","""BANAMEX""","""SG""","""Subasta Extraordinaria""",1750.0,5,"""1P""",3338,2011,"""Recup""","""Inicio""",0.0
0,"""GARANTIA EMPRESARIAL""","""NR""","""BANCO SANTANDER""","""SG""","""PyMe""",1690.0,4,"""Pari Passu""",35175,2015,"""Recup""","""Inicio""",0.0
0,"""GARANTIA EMPRESARIAL""","""NR""","""BANCO DEL BAJIO""","""CG""","""Tradicional""",1950.0,1,"""Pari Passu""",35145,2018,"""Recup""","""Inicio""",0.0
…,…,…,…,…,…,…,…,…,…,…,…,…,…
0,"""GARANTIA EMERGENTES""","""NR""","""BANCO SANTANDER""","""SG""","""Emergente Contingencia Sanitar…",1390.0,4,"""Pari Passu""",31015,2011,"""Recup""","""Inicio""",0.0
0,"""GARANTIA SECTORIAL""","""NR""","""BANCA MIFEL""","""SG""","""Taxi DF""",1930.0,4,"""Pari Passu""",3268,2011,"""Recup""","""Inicio""",0.0
0,"""GARANTIA EMPRESARIAL""","""R""","""HSBC MEXICO""","""SG""","""Micro Empresa PM""",1595.0,5,"""Pari Passu""",32907,2016,"""Recup""","""Inicio""",0.0


In [27]:
# TABLA_SEV_OBS_202504_Temp
tabla_sev_obs_temp = pagadas_agrup.filter(
    (pl.col("CICLOS_RESCAT") == 1) &
    (pl.col("ANIO(PAGO)") <= 2018)
).select([    # Seleccionar y transformar columnas
    pl.col("MM_UDIS"),
    pl.col("TAXONOMIA"),
    pl.col("NR_R"),
    pl.col("BANCO"),
    pl.col("CSG"),
    pl.col("AGRUPAMIENTO"),
    pl.col("AGRUPAMIENTO_ID"),
    pl.col("TIPGAR_CLAVE"),
    pl.col("ESQUEMA"),
    pl.col("TPRO_CLAVE"),
    pl.col("ANIO(PAGO)_New"),
    pl.lit("Pago").alias("CONCEPTO"),
    pl.lit("Inicio").alias("PERIODO"),
    pl.col("MPAGADO").alias("MONTO")
])

tabla_sev_obs_temp

MM_UDIS,TAXONOMIA,NR_R,BANCO,CSG,AGRUPAMIENTO,AGRUPAMIENTO_ID,TIPGAR_CLAVE,ESQUEMA,TPRO_CLAVE,ANIO(PAGO)_New,CONCEPTO,PERIODO,MONTO
i32,str,str,str,str,str,f64,i16,str,i32,i32,str,str,f64
0,"""GARANTIA EMPRESARIAL""","""NR""","""BANCO SANTANDER""","""SG""","""PyMe""",1690.0,4,"""Pari Passu""",31759,2015,"""Pago""","""Inicio""",-0.038288
0,"""GARANTIA EMPRESARIAL""","""NR""","""BBV BANCOMER""","""SG""","""PyMe""",1690.0,4,"""Pari Passu""",35005,2015,"""Pago""","""Inicio""",-0.146386
1,"""GARANTIA EMPRESARIAL""","""NR""","""BBV BANCOMER""","""CG""","""Empresas""",1480.0,1,"""Pari Passu""",3205,2013,"""Pago""","""Inicio""",-3.689824
0,"""GARANTIA SUBASTA""","""NR""","""GM FINANCIAL DE MEXICO SA DE C…","""CG""","""Subasta Diciembre 2013""",1845.0,3,"""1P""",32444,2015,"""Pago""","""Inicio""",-0.457342
0,"""GARANTIA EMPRESARIAL""","""R""","""BANCA AFIRME""","""SG""","""Tarjeta Credito Pyme""",2005.0,4,"""Pari Passu""",32494,2016,"""Pago""","""Inicio""",-0.418594
…,…,…,…,…,…,…,…,…,…,…,…,…,…
0,"""GARANTIA SECTORIAL""","""NR""","""BANCO SANTANDER""","""SG""","""Emprendedores Tradicional""",1470.0,4,"""Pari Passu""",31166,2013,"""Pago""","""Inicio""",-0.197204
0,"""GARANTIA EMPRESARIAL""","""R""","""BBV BANCOMER""","""SG""","""Tarjeta Negocios""",1920.0,4,"""Pari Passu""",35260,2017,"""Pago""","""Inicio""",-0.482677
0,"""GARANTIA EMPRESARIAL""","""NR""","""BANCO DEL BAJIO""","""CG""","""Tradicional""",1950.0,1,"""Pari Passu""",3028,2011,"""Pago""","""Inicio""",-1.258048


In [34]:
# TABLA_SEV_OBS_202504
tabla_sev_obs = pl.concat([tabla_sev_obs, tabla_sev_obs_temp.cast({'TIPGAR_CLAVE': pl.Int32})])

In [43]:
# Definir lista de TAXONOMIA a excluir
excluir_taxonomia = [
    "GARANTIAS BANCOMEXT",
    "GARANTIAS SHF/LI FINANCIERO",
    "GARANTIAS SHF-LI FINANCIERO",
    "GARANTIAS BANSEFI"
]

# PAGADAS_AGRUP_202504_VF
pagadas_vf = pagadas_agrup.filter(~pl.col("TAXONOMIA").is_in(excluir_taxonomia))

# RECUP_AGRUP_202504_VF
recup_vf = recup_agrup.filter(~pl.col("TAXONOMIA").is_in(excluir_taxonomia))

In [51]:
recup_vf

IND_ENTRA,MM_UDIS,TAXONOMIA,NR_R,BANCO,CSG,CSF,AGRUPAMIENTO,AGRUPAMIENTO_ID,CICLOS_RESCAT,DIFERENCIAS_MESES,TIPGAR_CLAVE,ANIO(PAGO),REEST,MRECUP_TOT,MRESCAT_TOT,Anio_REG_RECUP,ESQUEMA,ANIO(PAGO)_New,Intermediario_Id,TPRO_CLAVE,ORDEN_PAGO
i32,i32,str,str,str,str,str,str,f64,i32,i32,i32,i32,str,f64,f64,i32,str,i32,str,i32,i32
1,0,"""GARANTIA EMPRESARIAL""","""R""","""BANCO MERCANTIL DEL NORTE""","""SG""","""CF""","""Crediactivo Diana""",1275.0,1,27,5,2012,"""NE""",0.0,0.0,2015,"""Pari Passu""",2012,"""10040072""",31028,201212
1,0,"""GARANTIA EMPRESARIAL""","""R""","""HSBC MEXICO""","""SG""","""SF""","""Montos Menores Self Employ""",1635.0,1,150,5,2011,"""NE""",0.0,0.0,2024,"""Pari Passu""",2011,"""10000171""",3851,201106
1,0,"""GARANTIA SUBASTA""","""R""","""BANAMEX""","""SG""","""CF""","""Subasta Extraordinaria""",1750.0,1,44,5,2011,"""NE""",0.0,0.0,2014,"""1P""",2011,"""10000233""",3338,201104
1,0,"""GARANTIA EMPRESARIAL""","""R""","""BANAMEX""","""SG""","""CF""","""PyMe""",1690.0,1,41,5,2019,"""NE""",0.0,0.0,2022,"""Pari Passu""",2019,"""10000233""",35709,201905
1,0,"""GARANTIA EMPRESARIAL""","""NR""","""BANCO SANTANDER""","""SG""","""CF""","""PyMe""",1690.0,1,5,4,2015,"""NE""",0.0,0.0,2016,"""Pari Passu""",2015,"""10000144""",35175,201508
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
1,0,"""GARANTIA EMERGENTES""","""NR""","""BANCO SANTANDER""","""SG""","""CF""","""Emergente Contingencia Sanitar…",1390.0,1,48,4,2011,"""NE""",0.0,0.0,2015,"""Pari Passu""",2011,"""10000144""",31015,201105
1,0,"""GARANTIA SECTORIAL""","""NR""","""BANCA MIFEL""","""SG""","""CF""","""Taxi DF""",1930.0,1,2,4,2011,"""NE""",0.0,0.0,2012,"""Pari Passu""",2011,"""10000420""",3268,201112
1,0,"""GARANTIA EMPRESARIAL""","""R""","""HSBC MEXICO""","""SG""","""CF""","""Micro Empresa PM""",1595.0,1,2,5,2016,"""NE""",0.0,0.0,2017,"""Pari Passu""",2016,"""10000171""",32907,201611


##### Validadores

In [44]:
valida_pagos = (
    pagadas_vf
    .group_by("TAXONOMIA")
    .agg(pl.col("MPAGADO").sum().alias("S_MPagado"))
)
valida_pagos

TAXONOMIA,S_MPagado
str,f64
"""GARANTIA SECTORIAL""",-8639.926469
"""GARANTIA EMPRESARIAL""",-39222.382216
"""GARANTIA EMERGENTES""",-2219.294959
"""GARANTIA EMERGENCIAS""",-1.227871
"""EMPRESA MEDIANA""",-1529.779209
"""GARANTIA SUBASTA""",-8557.131859
"""GARANTIA FIANZA""",-7.096223
"""GARANTIAS FEFA""",-69.381424
"""GARANTIA MICROCREDITO""",-688.199388


In [45]:
valida_recup_rescat = (
    recup_vf
    .filter(pl.col("IND_ENTRA") == 1)
    .group_by("TAXONOMIA")
    .agg([
        pl.col("MRECUP_TOT").sum().alias("S_MRcup"),
        pl.col("MRESCAT_TOT").sum().alias("S_MRescat")
    ])
)
valida_recup_rescat

TAXONOMIA,S_MRcup,S_MRescat
str,f64,f64
"""EMPRESA MEDIANA""",0.0,0.0
"""GARANTIA SUBASTA""",0.0,0.0
"""GARANTIA SELECTIVA""",0.0,0.0
"""GARANTIA EMPRESARIAL""",0.0,0.0
"""GARANTIA MICROCREDITO""",0.0,0.0
"""GARANTIA SECTORIAL""",0.0,0.0
"""GARANTIA EMERGENTES""",0.0,0.0
"""GARANTIA FIANZA""",0.0,0.0


In [46]:
valida_pagos_fide = (
    pagadas_vf
    .filter(pl.col("AGRUPAMIENTO") == "FIDE")
    .group_by("TAXONOMIA")
    .agg(pl.col("MPAGADO").sum().alias("S_MPagado"))
)
valida_pagos_fide

TAXONOMIA,S_MPagado
str,f64
"""GARANTIA MICROCREDITO""",-469.87856


In [48]:
valida_recup_rescat_fide = (
    recup_vf
    .filter((pl.col("AGRUPAMIENTO") == "FIDE") & (pl.col("IND_ENTRA") == 1))
    .group_by("TAXONOMIA")
    .agg([
        pl.col("MRECUP_TOT").sum().alias("S_MRcup"),
        pl.col("MRESCAT_TOT").sum().alias("S_MRescat")
    ])
)
valida_recup_rescat

TAXONOMIA,S_MRcup,S_MRescat
str,f64,f64
"""EMPRESA MEDIANA""",0.0,0.0
"""GARANTIA SUBASTA""",0.0,0.0
"""GARANTIA SELECTIVA""",0.0,0.0
"""GARANTIA EMPRESARIAL""",0.0,0.0
"""GARANTIA MICROCREDITO""",0.0,0.0
"""GARANTIA SECTORIAL""",0.0,0.0
"""GARANTIA EMERGENTES""",0.0,0.0
"""GARANTIA FIANZA""",0.0,0.0
